In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations
import matplotlib.pyplot as plt
import math

# Input basic data

In [ ]:
# Define the number of sections, which is 6 here
num_of_sections = 6
num_of_days = 3  # Define the number of days, which is 3 here
# Define M
M = 1080
# Define the unscheduling penalty cost for maintenance activities with different importance level
importance_penalty_costs = {
    1: 1000,  # 1 is "normal", the unscheduling penalty cost is set to 1000
    2: 2000,  # 2 is "important", the unscheduling penalty cost is set to 2000
    3: 3000  # 3 is "very important", the unscheduling penalty cost is set to 3000
}

Modify the number of maintenance activities that can be scheduled simultaneously here

In [ ]:
max_compatible_activities = 3  # A maximum of 3 maintenance activities can be scheduled simultaneously in each maintenance window

Modify the maintenance volume and its distribution

In [ ]:
balanced_maintenance_indicator = 1  # 1 means relatively balanced distribution

In [ ]:
maintenance_volume_indicator = 2

In [ ]:
if maintenance_volume_indicator == 1:  # 1 means the maintenance volume of 180, and each section has 30 maintenance activities
    if balanced_maintenance_indicator == 1:  # 1 means relatively balanced distribution
        maintenance_csv = 'balanced_180_maintenance_volume.csv'  # Read the file of 'balanced_180_maintenance_volume.csv'
    else:
        maintenance_csv = 'unbalanced_180_maintenance_volume.csv'  # Read the file of 'unbalanced_180_maintenance_volume.csv'
else:  # 2 means the maintenance volume of 240, and each section has 40 maintenance activities
    if balanced_maintenance_indicator == 1:  # 1 means relatively balanced distribution
        maintenance_csv = 'balanced_240_maintenance_volume.csv'  # Read the file of 'balanced_240_maintenance_volume.csv'
    else:
        maintenance_csv = 'unbalanced_240_maintenance_volume.csv'  # Read the file of 'balanced_240_maintenance_volume.csv'

In [ ]:
maintenance_df = pd.read_csv(maintenance_csv)

# Create a dictionary 'maintenance_activities_info' to store detailed information for each maintenance activity
maintenance_activities_info = {}  # The key is the activity number, and the value is a list containing the activity type, the start day of the optimal scheduling range, the end day of the optimal scheduling range, the fixed scheduling day, the duration, and the importance level.

for _, row in maintenance_df.iterrows():
    maintenance_activities_info[int(row['Activity ID'])] = [
        int(row['Activity Type']),
        int(row['Start Date']),
        int(row['End Date']),
        int(row['Fixed Scheduling Date']),
        int(row['Estimated Duration']),
        int(row['Importance Level'])
    ]

# Create a dictionary 'section_day_maintenance_activities' to store the activities for each section on each day
section_day_maintenance_activities = {}

for section in range(1, num_of_sections + 1):
    section_maintenance_activities = list(
        maintenance_df[maintenance_df['Section'] == section]['Activity ID']
    )
    for day in range(1, 4):
        section_day_maintenance_activities[(section, day)] = []
        for activity_id in section_maintenance_activities:
            if maintenance_activities_info[activity_id][3] == 0 or maintenance_activities_info[activity_id][3] == day:
                section_day_maintenance_activities[(section, day)].append(activity_id)

        section_day_maintenance_activities[(section, day)].sort()

Modify the mutually exclusive relationship between maintenance activities here

In [ ]:
maintenance_mutually_exclusive_indicator = 2

In [ ]:
if maintenance_mutually_exclusive_indicator == 1:  # 1 means 29 mutually exclusive maintenance activity pairs
    mutually_exclusive_activities = {
        1: [16, 17, 2],
        3: [4, 8, 16, 17, 2],
        4: [3, 8, 16, 17],
        8: [3, 4, 16, 17],
        11: [16, 17, 14],
        12: [16, 17, 7],
        13: [16, 17, 15],
        15: [16, 17, 10, 13],
        16: [1, 3, 4, 8, 11, 12, 13, 15],
        17: [1, 3, 4, 8, 11, 12, 13, 15],
        2: [1, 3, 14],
        5: [6, 10],
        6: [5],
        7: [12],
        9: [10],
        10: [5, 9, 15],
        14: [2, 11]
    }  # The key is the activity type number, and the value is a list containing the activity type numbers that are mutually exclusive with the activity.
else:  # 2 means 39 mutually exclusive maintenance activity pairs
    mutually_exclusive_activities = {
        1: [16, 17, 2, 5],
        3: [4, 8, 16, 17, 2],
        4: [3, 8, 16, 17],
        8: [3, 4, 16, 17],
        11: [16, 17, 14, 9],
        12: [16, 17, 7, 15],
        13: [16, 17, 10, 15],
        15: [16, 17, 12, 13, 10],
        16: [1, 3, 4, 8, 11, 12, 13, 15],
        17: [1, 3, 4, 8, 11, 12, 13, 15, 14],
        2: [1, 3, 7, 14],
        5: [1, 6, 9, 10],
        6: [5, 7, 10],
        7: [2, 6, 9, 12],
        9: [5, 7, 10, 11],
        10: [5, 6, 9, 13, 15],
        14: [2, 11, 17]
    }

# Solve the maintenance scheduling model for each section of each day

In [ ]:
# By inputting the daily maintenance activity list for each section, obtain the list of maintenance activities actually scheduled within the maintenance window of that section on that day.
def obtain_actually_scheduled_maintenance_activities(input_maintenance_activity_ids,
                                                     input_maintenance_window_start_time,
                                                     input_maintenance_window_end_time):
    # Construct the model
    obtain_actually_scheduled_maintenance_activities_model = gp.Model(
        "obtain_actually_scheduled_maintenance_activities_model")

    # Define variables, their meanings are similar to those introduced in the file of 'experiment_with_different_daily_train_volumes_maintenance_volume_mutually_exclusion_max_compatible_num.ipynb'
    inner_maintenance_activity_scheduling_vars = {}
    inner_maintenance_activity_start_time_vars = {}
    inner_maintenance_activity_end_time_vars = {}
    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_scheduling_vars[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addVar(
            vtype=GRB.BINARY,
            name=f'inner_maintenance_activity_scheduling_var_activity_{inner_activity_id}')
        inner_maintenance_activity_start_time_vars[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addVar(
            lb=0,
            ub=input_maintenance_window_end_time,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_start_time_var_activity_{inner_activity_id}'
        )
        inner_maintenance_activity_end_time_vars[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addVar(
            lb=0,
            ub=input_maintenance_window_end_time,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_end_time_var_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_both_scheduling_indicator_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        for inner_activity_id_2 in input_maintenance_activity_ids:
            if inner_activity_id_1 < inner_activity_id_2:
                inner_maintenance_activities_both_scheduling_indicator_vars[(inner_activity_id_1,
                                                                             inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                    vtype=GRB.BINARY,
                    name=f'inner_maintenance_activities_both_scheduling_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                )

    inner_maintenance_activities_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1, inner_activity_id_2) in inner_maintenance_activities_seq_vars or (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_seq_vars:
                        continue
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_1,
                                                               inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activity_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_2,
                                                               inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activity_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )

    inner_maintenance_activities_scheduling_simultaneously_indicator_vars = {}
    inner_maintenance_activities_start_end_time_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 not in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                        if inner_activity_id_1 < inner_activity_id_2:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1,
                                 inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                            )
                        else:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_2,
                                 inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                            )
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
        else:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities:
                    if inner_activity_type_1 in mutually_exclusive_activities[inner_activity_type_2]:
                        continue
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_1,
                         inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                    )
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_2,
                         inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                    )

    # Define constraints, their meanings are similar to those introduced in the file of 'experiment_with_different_daily_train_volumes_maintenance_volume_mutually_exclusion_max_compatible_num.ipynb'
    inner_maintenance_activity_scheduling_range_cons = {}
    inner_maintenance_activity_duration_cons = {}

    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_duration = maintenance_activities_info[inner_activity_id][4]
        inner_start_time_var = inner_maintenance_activity_start_time_vars[inner_activity_id]
        inner_end_time_var = inner_maintenance_activity_end_time_vars[inner_activity_id]
        inner_x_var = inner_maintenance_activity_scheduling_vars[inner_activity_id]

        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var <= input_maintenance_window_end_time * inner_x_var,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var >= input_maintenance_window_start_time * inner_x_var,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_end_time_var <= input_maintenance_window_end_time * inner_x_var,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_end_time_var >= input_maintenance_window_start_time * inner_x_var,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_duration_cons[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_end_time_var - inner_start_time_var == inner_maintenance_activity_duration * inner_x_var,  # 持续时间
            name=f'inner_maintenance_activity_duration_con_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_both_scheduling_cons = {}
    for (inner_activity_id_1,
         inner_activity_id_2), inner_var in inner_maintenance_activities_both_scheduling_indicator_vars.items():
        inner_x_var_1 = inner_maintenance_activity_scheduling_vars[inner_activity_id_1]
        inner_x_var_2 = inner_maintenance_activity_scheduling_vars[inner_activity_id_2]
        inner_maintenance_activities_both_scheduling_cons[(inner_activity_id_1, inner_activity_id_2,
                                                           1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_var <= inner_x_var_1,
            name=f'inner_maintenance_activities_both_scheduling_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
        )
        inner_maintenance_activities_both_scheduling_cons[(inner_activity_id_1, inner_activity_id_2,
                                                           2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_var <= inner_x_var_2,
            name=f'inner_maintenance_activities_both_scheduling_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
        )
        inner_maintenance_activities_both_scheduling_cons[(inner_activity_id_1, inner_activity_id_2,
                                                           3)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_var >= inner_x_var_1 + inner_x_var_2 - 1,
            name=f'inner_maintenance_activities_both_scheduling_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_3'
        )

    inner_maintenance_activities_seq_cons = {}
    for (inner_activity_id_1, inner_activity_id_2), inner_var in inner_maintenance_activities_seq_vars.items():
        inner_start_time_var_1 = inner_maintenance_activity_start_time_vars[inner_activity_id_1]
        inner_start_time_var_2 = inner_maintenance_activity_start_time_vars[inner_activity_id_2]
        inner_end_time_var_1 = inner_maintenance_activity_end_time_vars[inner_activity_id_1]
        inner_end_time_var_2 = inner_maintenance_activity_end_time_vars[inner_activity_id_2]
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var_2 + M * (1 - inner_var) >= inner_end_time_var_1,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
        )
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var_1 + M * inner_var >= inner_end_time_var_2,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
        )

    inner_major_maintenance_activity_must_be_scheduled_cons = {}
    for inner_activity_id in input_maintenance_activity_ids:
        inner_activity_type = maintenance_activities_info[inner_activity_id][0]
        if inner_activity_type == 3 or inner_activity_type == 4 or inner_activity_type == 8:
            inner_major_maintenance_activity_must_be_scheduled_cons[
                inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                inner_maintenance_activity_scheduling_vars[inner_activity_id] == 1,
                name=f'inner_major_maintenance_activity_must_be_scheduled_con_activity_{inner_activity_id}'
            )

    inner_maintenance_activities_simultaneous_number_limitation_cons = {}
    for inner_activity_combination in combinations(input_maintenance_activity_ids, max_compatible_activities + 1):
        inner_activity_combination = sorted(inner_activity_combination)
        for inner_activity_id_1 in inner_activity_combination:
            for inner_activity_id_2 in inner_activity_combination:
                if inner_activity_id_2 > inner_activity_id_1:
                    if (inner_activity_id_1, inner_activity_id_2, 1) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            2) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            3) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            4) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            5) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            6) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons:
                        continue
                    if (inner_activity_id_1,
                        inner_activity_id_2) in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_1, inner_activity_id_2)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             3)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_2, inner_activity_id_1)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_3'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             4)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_4'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             5)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)] ==
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)] +
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)] - 1,
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_5'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             6)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)] <=
                            inner_maintenance_activities_both_scheduling_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_6'
                        )
        if tuple(inner_activity_combination) not in \
                inner_maintenance_activities_simultaneous_number_limitation_cons:
            inner_temp_expr = gp.LinExpr(0)
            for inner_activity_id_1 in inner_activity_combination:
                for inner_activity_id_2 in inner_activity_combination:
                    if inner_activity_id_2 > inner_activity_id_1:
                        if (inner_activity_id_1,
                            inner_activity_id_2) in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                            inner_temp_expr += \
                                inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                    (inner_activity_id_1, inner_activity_id_2)]
            inner_maintenance_activities_simultaneous_number_limitation_cons[
                tuple(inner_activity_combination)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                inner_temp_expr <= math.comb(max_compatible_activities + 1, 2) - 1,
                name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_combination}'
            )

    # Define the objective function
    inner_obj = gp.LinExpr(0)
    for inner_activity_id in input_maintenance_activity_ids:
        inner_importance_level = maintenance_activities_info[inner_activity_id][5]
        inner_x_var = inner_maintenance_activity_scheduling_vars[inner_activity_id]
        inner_obj += importance_penalty_costs[inner_importance_level] * inner_x_var
    obtain_actually_scheduled_maintenance_activities_model.setObjective(inner_obj, GRB.MAXIMIZE)  # The objective is to maximize

    obtain_actually_scheduled_maintenance_activities_model.setParam('OutputFlag', 0)
    obtain_actually_scheduled_maintenance_activities_model.setParam('TimeLimit', 60)
    obtain_actually_scheduled_maintenance_activities_model.setParam('MIPGap', 0.01)

    # Solve the model
    obtain_actually_scheduled_maintenance_activities_model.optimize()

    # Obtain a list of maintenance activities that are actually scheduled
    output_actually_scheduled_maintenance_activities = []
    for inner_activity_id, inner_x_var in inner_maintenance_activity_scheduling_vars.items():
        if inner_x_var.X > 0.5:
            output_actually_scheduled_maintenance_activities.append(inner_activity_id)

    return list(sorted(
        set(output_actually_scheduled_maintenance_activities)))

In [ ]:
# By inputting the list of maintenance activities actually scheduled every day in each section, the shortest maintenance window duration that can be achieved are the specific time for scheduling maintenance activities under the current duration, is obtained.
def obtain_actually_scheduling_plan(input_maintenance_activity_ids,
                                    input_maintenance_window_start_time,
                                    input_maintenance_window_end_time):
    # Construct the model
    obtain_actually_scheduling_plan_model = gp.Model(
        "obtain_actually_scheduling_plan_model")

    # Define variables, their meanings are similar to those introduced in the file of 'experiment_with_different_daily_train_volumes_maintenance_volume_mutually_exclusion_max_compatible_num.ipynb'
    inner_maintenance_activity_start_time_vars = {}
    inner_maintenance_activity_end_time_vars = {}
    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_start_time_vars[
            inner_activity_id] = obtain_actually_scheduling_plan_model.addVar(
            lb=input_maintenance_window_start_time,
            ub=input_maintenance_window_end_time,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_start_time_var_activity_{inner_activity_id}'
        )
        inner_maintenance_activity_end_time_vars[
            inner_activity_id] = obtain_actually_scheduling_plan_model.addVar(
            lb=input_maintenance_window_start_time,
            ub=input_maintenance_window_end_time,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_end_time_var_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1, inner_activity_id_2) in inner_maintenance_activities_seq_vars or (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_seq_vars:
                        continue
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_1,
                                                               inner_activity_id_2)] = obtain_actually_scheduling_plan_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_2,
                                                               inner_activity_id_1)] = obtain_actually_scheduling_plan_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )

    inner_maintenance_activities_scheduling_simultaneously_indicator_vars = {}
    inner_maintenance_activities_start_end_time_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 not in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                        if inner_activity_id_1 < inner_activity_id_2:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1,
                                 inner_activity_id_2)] = obtain_actually_scheduling_plan_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                            )
                        else:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_2,
                                 inner_activity_id_1)] = obtain_actually_scheduling_plan_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                            )
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = obtain_actually_scheduling_plan_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = obtain_actually_scheduling_plan_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
        else:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities:
                    if inner_activity_type_1 in mutually_exclusive_activities[inner_activity_type_2]:
                        continue
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = obtain_actually_scheduling_plan_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = obtain_actually_scheduling_plan_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_1,
                         inner_activity_id_2)] = obtain_actually_scheduling_plan_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                    )
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_2,
                         inner_activity_id_1)] = obtain_actually_scheduling_plan_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                    )

    inner_maintenance_window_duration_var = obtain_actually_scheduling_plan_model.addVar(
        lb=0,
        ub=M,
        vtype=GRB.CONTINUOUS,
        name='inner_maintenance_window_duration_var'
    )

    # Define the constraints, their meanings are similar to those introduced in the file of 'experiment_with_different_daily_train_volumes_maintenance_volume_mutually_exclusion_max_compatible_num.ipynb'
    inner_maintenance_activity_scheduling_range_cons = {}
    inner_maintenance_activity_duration_cons = {}

    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_duration = maintenance_activities_info[inner_activity_id][4]
        inner_start_time_var = inner_maintenance_activity_start_time_vars[inner_activity_id]
        inner_end_time_var = inner_maintenance_activity_end_time_vars[inner_activity_id]

        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 1)] = obtain_actually_scheduling_plan_model.addConstr(
            inner_start_time_var <= input_maintenance_window_start_time + inner_maintenance_window_duration_var,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 2)] = obtain_actually_scheduling_plan_model.addConstr(
            inner_start_time_var >= input_maintenance_window_start_time,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 1)] = obtain_actually_scheduling_plan_model.addConstr(
            inner_end_time_var <= input_maintenance_window_start_time + inner_maintenance_window_duration_var,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 2)] = obtain_actually_scheduling_plan_model.addConstr(
            inner_end_time_var >= input_maintenance_window_start_time,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_duration_cons[
            inner_activity_id] = obtain_actually_scheduling_plan_model.addConstr(
            inner_end_time_var - inner_start_time_var == inner_maintenance_activity_duration,
            name=f'inner_maintenance_activity_duration_con_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_seq_cons = {}
    for (inner_activity_id_1, inner_activity_id_2), inner_var in inner_maintenance_activities_seq_vars.items():
        inner_start_time_var_1 = inner_maintenance_activity_start_time_vars[inner_activity_id_1]
        inner_start_time_var_2 = inner_maintenance_activity_start_time_vars[inner_activity_id_2]
        inner_end_time_var_1 = inner_maintenance_activity_end_time_vars[inner_activity_id_1]
        inner_end_time_var_2 = inner_maintenance_activity_end_time_vars[inner_activity_id_2]
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               1)] = obtain_actually_scheduling_plan_model.addConstr(
            inner_start_time_var_2 + M * (1 - inner_var) >= inner_end_time_var_1,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
        )
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               2)] = obtain_actually_scheduling_plan_model.addConstr(
            inner_start_time_var_1 + M * inner_var >= inner_end_time_var_2,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
        )

    inner_maintenance_activities_simultaneous_number_limitation_cons = {}
    for inner_activity_combination in combinations(input_maintenance_activity_ids, max_compatible_activities + 1):
        inner_activity_combination = sorted(inner_activity_combination)
        for inner_activity_id_1 in inner_activity_combination:
            for inner_activity_id_2 in inner_activity_combination:
                if inner_activity_id_2 > inner_activity_id_1:
                    if (inner_activity_id_1, inner_activity_id_2, 1) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            2) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            3) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            4) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            5) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            6) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons:
                        continue
                    if (inner_activity_id_1,
                        inner_activity_id_2) in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             1)] = obtain_actually_scheduling_plan_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_1, inner_activity_id_2)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             2)] = obtain_actually_scheduling_plan_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             3)] = obtain_actually_scheduling_plan_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_2, inner_activity_id_1)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_3'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             4)] = obtain_actually_scheduling_plan_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_4'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             5)] = obtain_actually_scheduling_plan_model.addConstr(
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)] ==
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)] +
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)] - 1,
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_5'
                        )
        if tuple(inner_activity_combination) not in \
                inner_maintenance_activities_simultaneous_number_limitation_cons:
            inner_temp_expr = gp.LinExpr(0)
            for inner_activity_id_1 in inner_activity_combination:
                for inner_activity_id_2 in inner_activity_combination:
                    if inner_activity_id_2 > inner_activity_id_1:
                        if (inner_activity_id_1,
                            inner_activity_id_2) in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                            inner_temp_expr += \
                                inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                    (inner_activity_id_1, inner_activity_id_2)]
            inner_maintenance_activities_simultaneous_number_limitation_cons[
                tuple(
                    inner_activity_combination)] = obtain_actually_scheduling_plan_model.addConstr(
                inner_temp_expr <= math.comb(max_compatible_activities + 1, 2) - 1,
                name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_combination}'
            )

    # Define the objective function
    obtain_actually_scheduling_plan_model.setObjective(
        inner_maintenance_window_duration_var, GRB.MINIMIZE)  # The objective is to minimize

    obtain_actually_scheduling_plan_model.setParam('OutputFlag', 0)
    obtain_actually_scheduling_plan_model.setParam('TimeLimit', 60)
    obtain_actually_scheduling_plan_model.setParam('MIPGap', 0.01)

    # Solve the model
    obtain_actually_scheduling_plan_model.optimize()

    # Obtain the start time and end time of each maintenance activity
    output_maintenance_activity_start_time_dict = {
        inner_activity_id: int(inner_maintenance_activity_start_time_vars[inner_activity_id].X)
        for inner_activity_id in input_maintenance_activity_ids
    }
    output_maintenance_activity_end_time_dict = {
        inner_activity_id: int(inner_maintenance_activity_end_time_vars[inner_activity_id].X)
        for inner_activity_id in input_maintenance_activity_ids
    }
    return output_maintenance_activity_start_time_dict, output_maintenance_activity_end_time_dict  # Return the start time and end time of each maintenance activity

In [ ]:
obj2_val = 0
# Initialize the number of unscheduled maintenance activities at each importance level
importance_unscheduled_activities_num = {importance_level: 0 for importance_level in importance_penalty_costs.keys()}
# Initialize the actually scheduling time of scheduled maintenance activities for each section and each day
section_day_maintenance_activity_start_time_dict = {(section, day): {} for section in range(1, num_of_sections + 1) for
                                                    day in
                                                    range(1, num_of_days + 1)}
section_day_maintenance_activity_end_time_dict = {(section, day): {} for section in range(1, num_of_sections + 1) for
                                                  day in range(1, num_of_days + 1)}

In [ ]:
# For each section and each day, obtain the list of actually scheduled maintenance activities, and on this basis obtain the shortest maintenance window duration and the specific time for scheduling maintenance activities under the current duration.
for section in range(1, num_of_sections + 1):
    for day in range(1, num_of_days + 1):
        all_activities = section_day_maintenance_activities[(section, day)]
        # Obtain the list of actually scheduled maintenance activities
        actually_scheduled_maintenance_activities = obtain_actually_scheduled_maintenance_activities(all_activities, 0,
                                                                                                     180)
        # Obtain the list of unscheduled maintenance activities
        unscheduled_maintenance_activities = list(
            set(all_activities) - set(actually_scheduled_maintenance_activities))
        print(f'The list of actually scheduled maintenance activities for section {section} and day {day} has been found. They are {actually_scheduled_maintenance_activities}, with a total of {len(actually_scheduled_maintenance_activities)} maintenance activities. The list of unscheduled maintenance activities is {unscheduled_maintenance_activities}, with a total of {len(unscheduled_maintenance_activities)} maintenance activities.')

        if len(unscheduled_maintenance_activities) > 0:
            for unscheduled_activity in unscheduled_maintenance_activities:
                importance_level = maintenance_activities_info[unscheduled_activity][5]
                importance_unscheduled_activities_num[importance_level] += 1
                obj2_val += importance_penalty_costs[importance_level]

        # Obtain the shortest achievable maintenance window duration and the specific time for scheduling maintenance activities under the current duration
        maintenance_activity_start_time_dict, maintenance_activity_end_time_dict = obtain_actually_scheduling_plan(
            actually_scheduled_maintenance_activities, 0, 180)
        section_day_maintenance_activity_start_time_dict[(section, day)] = maintenance_activity_start_time_dict
        section_day_maintenance_activity_end_time_dict[(section, day)] = maintenance_activity_end_time_dict
        print(f'The actual maintenance scheduling plan for section {section} and day {day} has been generated.')

In [ ]:
print(f'The objective function value is {obj2_val}.')
print(f'The number of unscheduled maintenance activities at each importance level is {importance_unscheduled_activities_num}.')

In [ ]:
section_heights = [78, 67.9, 67.9, 67.5, 75.6, 50.8]
width = 180

fig, axs = plt.subplots(6, 3, figsize=(18, 12),
                        gridspec_kw={
                            'height_ratios': section_heights[::-1],
                            'hspace': 0.3,
                            'wspace': 0.1
                        })

for section in range(1, num_of_sections + 1):
    for day in range(1, num_of_days + 1):
        ax = axs[6 - section, day - 1]

        for spine in ax.spines.values():
            spine.set_linewidth(3)
            spine.set_color('darkgray')

        ax.set_xlim(0, width)
        ax.set_ylim(0, section_heights[section - 1])

        ax.set_xticks([0, 180])
        ax.set_xticklabels(['0', '180'], fontsize=9)

        # 隐藏y轴
        ax.set_yticks([])

        # 设置标题
        ax.set_title(f'Section {section} - Day {day}', fontsize=10, pad=3)

        key = (section, day)
        start_dict = section_day_maintenance_activity_start_time_dict.get(key, {})
        end_dict = section_day_maintenance_activity_end_time_dict.get(key, {})

        for i, (activity_id, start_time) in enumerate(start_dict.items()):
            end_time = end_dict.get(activity_id)
            maintenance_info = maintenance_activities_info[activity_id]
            activity_type = maintenance_info[0]
            if activity_type == 3 or activity_type == 4 or activity_type == 8:
                color = 'yellow'
            else:
                color = 'saddlebrown'
            ax.plot([start_time, end_time],
                    [0, section_heights[section - 1]], color=color, linewidth=2, alpha=0.8)

plt.subplots_adjust(hspace=0.4, wspace=0.2)

if balanced_maintenance_indicator == 1:
    file_name_for_whether_balanced = "balanced"
else:
    file_name_for_whether_balanced = "unbalanced"

if balanced_maintenance_indicator == 1:
    file_name_for_whether_balanced = "balanced"
else:
    file_name_for_whether_balanced = "unbalanced"

if maintenance_volume_indicator == 1:
    file_name_for_maintenance_volume = '180_maintenance_volume'
else:
    file_name_for_maintenance_volume = '240_maintenance_volume'

if maintenance_mutually_exclusive_indicator == 1:
    file_name_for_mutually_exclusion = '29_mutually_exclusion'
else:
    file_name_for_mutually_exclusion = '39_mutually_exclusion'

# 保存矢量图（SVG格式）
plt.savefig(
    f'maintenance_scheduling_plan_of_experiment_for_fixed_maintenance_scheduling.svg',
    format='svg', dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
# Calculate the utilization rate of maintenance window in each section and each day
section_day_maintenance_window_utilization_rate = {}
for section in range(1, num_of_sections + 1):
    for day in range(1, num_of_days + 1):
        latest_end_time = 0
        for activity_id in section_day_maintenance_activity_end_time_dict[(section, day)]:
            end_time = section_day_maintenance_activity_end_time_dict[(section, day)][activity_id]
            if end_time > latest_end_time:
                latest_end_time = end_time
        section_day_maintenance_window_utilization_rate[(day, section)] = round(latest_end_time / 180, 4)

In [ ]:
average_utilization_rate = sum(section_day_maintenance_window_utilization_rate.values()) / len(
    section_day_maintenance_window_utilization_rate)
print(f'Average utilization rate: {average_utilization_rate:.4f}')